<a href="https://colab.research.google.com/github/carlcodecode/Data-Science-II/blob/main/PracticeForTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets

transform = transforms.Compose([transforms.ToTensor()]) #Convert data to tensor
train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.80MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 124kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.18MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.90MB/s]


In [2]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

#splitting the data into managable mini-batches

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_load = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [3]:
#initializing the model

class SimpleNN(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(input_dim, 128)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(128, output_dim)

  def forward(self, x):
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x


In [4]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up epochs and learning rate
epochs = 5
lr = 0.01

# Set up model
model = SimpleNN(input_dim=28*28, output_dim=10).to(device)

# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
#Training loop

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for X_batch, y_batch in train_loader:   # iterate over the data loader

    X_batch, y_batch = X_batch.to(device), y_batch.to(device)

    # forward pass
    logits = model(X_batch)

    # calculate loss
    loss = loss_fn(logits, y_batch) # like loss y_hat vs y_true

    # 3 step mantra
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {epoch+1}: Loss = {running_loss / len(train_loader)}")

Epoch 1: Loss = 0.10857722012406544
Epoch 2: Loss = 0.09910962501408478
Epoch 3: Loss = 0.09240109827323531
Epoch 4: Loss = 0.08994934585498103
Epoch 5: Loss = 0.07726363078059345


In [9]:
model.eval() #set to evaluation mode

correct = 0
total = 0

with torch.no_grad():   #disable gradient calc
  for X_batch, y_batch in test_load:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    outputs = model(X_batch)

    _, predicted = torch.max(outputs.data, dim=1)
    total += y_batch.size(dim=0) #shape of model is (BATCH_SIZE, num_classes)
    correct += (predicted == y_batch).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of model: {accuracy}")


Accuracy of model: 98.135
